# Victor Bandhaso
# 2209106063


#impor modul file dari google.colab Kemudian jalankan perintah berikut untuk membuka dialog file unggah:


In [2]:
from google.colab import files


In [6]:
uploaded = files.upload()

Saving netflix_titles.csv to netflix_titles.csv


#membaca dataset CSV tersebut menggunakan pandas.

In [7]:
import pandas as pd

# Baca file CSV
df = pd.read_csv('netflix_titles.csv')
print(df.head())


  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

# Preprocessing Data
## Kita akan memfokuskan model pada prediksi apakah tayangan adalah Movie atau TV Show (kolom type).
##Fitur yang bisa kita gunakan adalah release_year, duration, rating, listed_in, dan lain-lain.
## Ubah kolom yang bertipe string menjadi numerik (misalnya dengan LabelEncoder).




In [23]:
from sklearn.preprocessing import LabelEncoder

# Menghapus baris yang memiliki nilai kosong
df = df.dropna()

# Mengubah 'duration' menjadi numerik
def convert_duration(duration):
    if 'Season' in duration:
        return int(duration.split()[0]) * 60  # Misalkan 1 season = 60 menit
    else:
        return int(duration.split()[0])  # Durasi dalam menit

df['duration'] = df['duration'].apply(convert_duration)

# Mengonversi kolom non-numerik menjadi numerik menggunakan LabelEncoder
label_encoders = {}
for column in ['type', 'rating', 'listed_in', 'country', 'director']:
    if column in df.columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le
    else:
        print(f"Kolom {column} tidak ditemukan dalam dataset.")

# Memilih fitur dan target
X = df[['release_year', 'rating', 'listed_in', 'country', 'duration']]
y = df['type']  # 0 untuk 'Movie', 1 untuk 'TV Show'

# Normalisasi data fitur
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset menjadi training dan testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Membangun Model Deep Learning Sederhana
## Kita akan menggunakan Sequential model dengan beberapa layer dense (fully connected).

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Membangun model Sequential
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    Dense(32, activation='relu'),  # Hidden layer
    Dense(1, activation='sigmoid')  # Output layer untuk prediksi biner
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Melatih model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8533 - loss: 0.4114 - val_accuracy: 0.9707 - val_loss: 0.1701
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9739 - loss: 0.1529 - val_accuracy: 0.9707 - val_loss: 0.1430
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9734 - loss: 0.1159 - val_accuracy: 0.9719 - val_loss: 0.1163
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9768 - loss: 0.0921 - val_accuracy: 0.9730 - val_loss: 0.1060
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9768 - loss: 0.0883 - val_accuracy: 0.9719 - val_loss: 0.1012
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9734 - loss: 0.0868 - val_accuracy: 0.9695 - val_loss: 0.0984
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9752 - loss: 0.0792 - val_accuracy: 0.9672 - val_loss: 0.0974
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9758 - loss: 0.0770 - val_accuracy: 0.9672 - val_

# Evaluasi Model
## Setelah model dilatih, kita akan mengevaluasi akurasinya pada data testing.


In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer

# ... (Your existing code for data preprocessing and feature scaling) ...

# Convert y_train and y_test to one-hot encoded vectors
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)


# Modify the output layer to match the number of classes
num_classes = y_train.shape[1]  # Get the number of classes from y_train shape
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')  # Output layer for multi-class
])

# Compile the model with categorical_crossentropy loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 266ms/step - accuracy: 0.2057 - loss: 1.1567 - val_accuracy: 0.5000 - val_loss: 1.0626
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3867 - loss: 1.0747 - val_accuracy: 0.6250 - val_loss: 1.0003
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6224 - loss: 1.0119 - val_accuracy: 0.7917 - val_loss: 0.9410
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8802 - loss: 0.9416 - val_accuracy: 0.7917 - val_loss: 0.8830
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8125 - loss: 0.8966 - val_accuracy: 0.7917 - val_loss: 0.8309
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8477 - loss: 0.8376 - val_accuracy: 0.7917 - val_loss: 0.7822
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8190 - loss: 0.7883 - val_accuracy: 0.7917 - val_loss: 0.7380
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8503 - loss: 0.7278 - val_accuracy: 0.7917 - val_loss: 0.6953